In [ ]:
import json
import pandas as pd
import requests
import zipfile
import io
import os
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

# Substitua pelos valores reais obtidos no Passo 1
account_name = 'uvabr'
account_key = os.environ['ACCOUNT_KEY']
container_name = 'bronze'

# Conecte-se ao serviço de armazenamento de blob
connect_str = f"DefaultEndpointsProtocol=https;AccountName={account_name};AccountKey={account_key};EndpointSuffix=core.windows.net"
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

# Acesse o contêiner
container_client = blob_service_client.get_container_client(container_name)

# Lista de anos desejados
anos = ['2019','2020','2021','2022', '2023', '2024']

# Lista de endpoints
endpoints = [
#    "proposicoes",
#    "proposicoesTemas",
#    "proposicoesAutores",
#    "frentes",
#    "frentesDeputados",
#    "grupos",
#    "gruposMembros",
#    "gruposHistorico",
#    "legislaturas",
#    "legislaturasMesas",
#    "orgaos",
#    "orgaosDeputados",
    "deputados",
#    "deputadosOcupacoes",
#    "deputadosProfissoes",
#    "eventos",
#    "eventosOrgaos",
#    "eventosRequerimentos",
#    "votacoes",
#    "votacoesOrientacoes",
#    "votacoesVotos",
#    "votacoesObjetos",
#    "votacoesProposicoes",
#    "funcionarios",
#    "licitacoes",
#    "licitacoesContratos",
#    "licitacoesItens",
#    "licitacoesPedidos",
#    "licitacoesPropostas",
#    "tecadTermos",
#    "tecadCategorias"
]

# Lista de endpoints exceções
endpointsExceptions = [
#    "frentes",
#    "frentesDeputados",
#    "grupos",
#    "gruposMembros",
#    "gruposHistorico",
#    "legislaturas",
#    "legislaturasMesas",
#    "orgaos",
    "deputados",
#    "deputadosOcupacoes",
#    "deputadosProfissoes",
#    "funcionarios",
#    "tecadTermos",
#    "tecadCategorias",
#    "orgaosDeputados"
]

# Função para obter os dados de uma URL específica
def obter_dados(url_completa):
    try:
        response = requests.get(url_completa)
        response.raise_for_status()  # Isso irá lançar uma exceção se a requisição falhar
        print(url_completa)
        data = response.json()
        return data.get('dados', [])
    except requests.exceptions.RequestException as e:
        print(f"Erro ao obter dados da URL {url_completa}: {e}")
        return []  # Retorna uma lista vazia em caso de erro

# Iterando sobre cada endpoint
for endpoint in endpoints:
    formato = 'json'  # ou qualquer outro formato que você deseja
    dados_concatenados = pd.DataFrame()
    # Verificando se o endpoint requer iteração sobre os anos
    if endpoint not in endpointsExceptions:
        # Iterando sobre cada ano
        for ano in anos:
            url = f'http://dadosabertos.camara.leg.br/arquivos/{endpoint}/{formato}/{endpoint}-{ano}.{formato}'
            dados_ano = obter_dados(url)
            if dados_ano:
                dados_ano = pd.json_normalize(dados_ano, max_level=1)
                dados_concatenados = pd.concat([dados_concatenados, dados_ano], ignore_index=True)
    elif endpoint == "orgaosDeputados":
        # Tratamento especial para o endpoint "orgaosDeputados"
        for legislatura in range(51, 58):  # Legislativas de 51 a 57
            url = f'http://dadosabertos.camara.leg.br/arquivos/{endpoint}/{formato}/{endpoint}-L{legislatura}.{formato}'
            dados_legislatura = obter_dados(url)
            print(url)
            if dados_legislatura:
                dados_legislatura = pd.json_normalize(dados_legislatura, max_level=1)
                dados_legislatura['legislatura'] = legislatura
                dados_concatenados = pd.concat([dados_concatenados, dados_legislatura], ignore_index=True)
    else:
        # Tratamento para endpoints que não possuem ano
        url = f'http://dadosabertos.camara.leg.br/arquivos/{endpoint}/{formato}/{endpoint}.{formato}'
        dados_ano = obter_dados(url)
        if dados_ano:
            dados_ano = pd.json_normalize(dados_ano, max_level=1)
            dados_concatenados = pd.concat([dados_concatenados, dados_ano], ignore_index=True)
            
    if not dados_concatenados.empty:
            # Obtendo o nome de referência do arquivo Parquet
        blob_name = f'{endpoint}.parquet'
        # Convertendo DataFrame para bytes
        parquet_bytes = dados_concatenados.to_parquet()
        # Fazendo upload diretamente para o Blob Storage
        container_client.upload_blob(name=blob_name, data=parquet_bytes, overwrite=True)
        print(f'Dados salvos como {blob_name} no Blob Storage')
    else:
        print(f'Não foram obtidos dados para o endpoint: {endpoint}')
        
def extract_json_from_zip(zip_content):
    with zipfile.ZipFile(io.BytesIO(zip_content)) as zip_ref:
        # Procura o arquivo JSON dentro do zip
        json_file = [f for f in zip_ref.namelist() if f.endswith('.json')][0]
        
        # Lê o conteúdo do arquivo JSON
        json_content = zip_ref.read(json_file)
        
        return json_content

def json_to_loads(json_content, parquet_file):
    # Decodifica o conteúdo JSON em uma string
    json_str = json_content.decode('utf-8')
    
    # Transformando em DataFrame
    df = pd.DataFrame(json.loads(json_str)['dados'])
    
    # Retorna o DataFrame
    return df

def process_api_data(years):
    # Lista para armazenar todos os DataFrames de todos os anos
    all_dfs = []
    
    for year in years:
        # URL da API da Câmara dos Deputados
        api_url = f"https://www.camara.leg.br/cotas/Ano-{year}.json.zip"
        
        # Faz o download do arquivo zip da URL
        response = requests.get(api_url)
        
        # Extrai o conteúdo do arquivo zip em memória
        json_content = extract_json_from_zip(response.content)
        # Converte o conteúdo JSON em DataFrame
        df = json_to_loads(json_content, f"Ano-{year}.parquet")
        # Adiciona o DataFrame à lista
        all_dfs.append(df)
    
    # Concatena todos os DataFrames em um único DataFrame
    combined_df = pd.concat(all_dfs, ignore_index=True)
    
    # Salva o DataFrame combinado como um único arquivo Parquet
    combined_df = combined_df.convert_dtypes()
    #combined_df.to_parquet(f"{diretorio_destino}/deputadosDespesas.parquet")
    blob_name = f'deputadosDespesas.parquet'
    parquet_bytes = combined_df.to_parquet()
    container_client.upload_blob(name=blob_name, data=parquet_bytes, overwrite=True)
    print("Dados de todos os anos das despesasDeputados convertidos para Parquet no Blob Storage com sucesso!")

# Chama a função para processar os dados de todos os anos e salvar em um único arquivo Parquet
process_api_data(anos)

